# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- hoursPerWindow: 4


# Preproccess

## read data

In [3]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [4]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [5]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 917
ph 954
po2 917
albumin 406
hba1c 326
lymphocyte 446
height 415
urine-ketone 294
crp 19


In [6]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1206

## split patients

In [7]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [8]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,rr,saps2,sbp,scr,sodium,sofa,use_NaHCO3,uti,wbc,weight
0,14866589,20066152,33060916,False,NaN,37,NaN,NaN,NaN,NaN,...,NaN,22,NaN,NaN,133.0,2,0.0,0,NaN,NaN
0,17922352,20070680,39895564,False,NaN,47,NaN,NaN,NaN,NaN,...,NaN,26,NaN,NaN,145.0,5,0.0,0,NaN,NaN
0,14849360,20152551,32817197,False,NaN,59,NaN,NaN,NaN,NaN,...,NaN,31,NaN,NaN,145.0,7,0.0,0,NaN,NaN
0,16815664,20240670,37751533,False,NaN,42,NaN,NaN,NaN,NaN,...,NaN,27,NaN,NaN,138.0,2,NaN,0,NaN,43.00
0,16792064,20244311,33026932,False,NaN,46,NaN,NaN,NaN,NaN,...,NaN,19,NaN,NaN,131.0,0,0.0,0,NaN,68.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,15517352,29801212,32814591,False,NaN,20,NaN,NaN,NaN,NaN,...,NaN,30,NaN,NaN,139.0,4,NaN,0,NaN,NaN
0,10383045,29899941,39755932,False,NaN,58,NaN,NaN,NaN,NaN,...,NaN,23,NaN,NaN,136.0,1,0.0,0,NaN,59.40
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,NaN,32,NaN,NaN,134.0,3,0.0,0,NaN,47.40
0,10912213,29911812,34040470,False,NaN,47,NaN,NaN,NaN,NaN,...,NaN,23,NaN,NaN,136.0,2,0.0,0,NaN,89.05


In [9]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [10]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

964 242
965 241
965 241
965 241
965 241


In [11]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [12]:
import pickle


(TEMP_PATH / f"train-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

11642159

In [13]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


datavalset = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    datavalset.append((npTrainX, trainY, npValX, valY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [14]:
import pickle

(TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(datavalset)
)

11468745

# LSTM

In [15]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC", "accuracy", "precision", "recall"])
    return model

2024-06-07 23:13:35.806948: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-07 23:13:35.825246: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-07 23:13:36.264387: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


2024-06-07 23:13:36.470020: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-07 23:13:36.485446: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/tu/codepy/hust.year2023.PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer usi

 1/25 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - AUC: 0.6437 - accuracy: 0.5938 - loss: 0.6762 - precision: 0.5000 - recall: 0.5385

25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - AUC: 0.6660 - accuracy: 0.6165 - loss: 0.6617 - precision: 0.5119 - recall: 0.5851 - val_AUC: 0.7718 - val_accuracy: 0.6839 - val_loss: 0.5990 - val_precision: 0.5567 - val_recall: 0.7500


Epoch 2/50


...

 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - AUC: 0.8913 - accuracy: 0.8750 - loss: 0.4044 - precision: 0.6923 - recall: 1.0000

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.9211 - accuracy: 0.8697 - loss: 0.3485 - precision: 0.7893 - recall: 0.8987 - val_AUC: 0.6807 - val_accuracy: 0.6736 - val_loss: 0.7840 - val_precision: 0.5616 - val_recall: 0.5694


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.8086 - accuracy: 0.7500 - loss: 0.5559 - precision: 0.7500 - recall: 0.7500

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step - AUC: 0.8026 - accuracy: 0.7204 - loss: 0.5499 - precision: 0.6294 - recall: 0.7994


Loses: [0.5958143472671509, 0.5677422881126404, 0.5178367495536804, 0.5827284455299377, 0.5630064010620117] 0.5654256463050842 0.026452911665840947
AUCs: [0.751987099647522, 0.8025039434432983, 0.848594069480896, 0.7599495053291321, 0.781470775604248] 0.7889010787010193 0.03465353064789468
Accuracies: [0.7190082669258118, 0.7427386045455933, 0.7302904725074768, 0.7053942084312439, 0.7095435857772827] 0.7213950276374816 0.013688365357476048
Precisions: [0.6336633563041687, 0.640350878238678, 0.6190476417541504, 0.6071428656578064, 0.5984252095222473] 0.6197259902954102 0.015698118472413796
Recals: [0.6736842393875122, 0.7765957713127136, 0.821052610874176, 0.7157894968986511, 0.800000011920929] 0.7574244260787963 0.054725959257304314


## Static validate 

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )
    return model

In [19]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

if "datavalset" not in locals():
    datavalset = pickle.loads(
        (TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )

loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npValX, valY, npTestX, testY) in enumerate(datavalset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    valY = np.array(valY)
    testY = np.array(testY)

    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=(npValX, valY),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - AUC: 0.5792 - accuracy: 0.5000 - loss: 0.6789 - precision: 0.4091 - recall: 0.7500

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - AUC: 0.6476 - accuracy: 0.5535 - loss: 0.6741 - precision: 0.4680 - recall: 0.7835 - val_AUC: 0.7891 - val_accuracy: 0.7292 - val_loss: 0.6300 - val_precision: 0.6211 - val_recall: 0.7867


Epoch 2/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - AUC: 0.8208 - accuracy: 0.8125 - loss: 0.5672 - precision: 0.7143 - recall: 0.8333

...

 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - AUC: 0.9844 - accuracy: 0.9062 - loss: 0.2018 - precision: 0.7778 - recall: 0.8750

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.9486 - accuracy: 0.8612 - loss: 0.3002 - precision: 0.7813 - recall: 0.8874 - val_AUC: 0.7830 - val_accuracy: 0.7254 - val_loss: 0.6990 - val_precision: 0.6575 - val_recall: 0.6316


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.8047 - accuracy: 0.7812 - loss: 0.5244 - precision: 0.7647 - recall: 0.8125

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8118 - accuracy: 0.7278 - loss: 0.5450 - precision: 0.6364 - recall: 0.8022


Loses: [0.5681694746017456, 0.5378271341323853, 0.4915633499622345, 0.5816780924797058, 0.5706989765167236] 0.549987405538559 0.03263464085066993
AUCs: [0.7646974921226501, 0.8011289834976196, 0.8422855138778687, 0.7781543135643005, 0.7922855019569397] 0.7957103610038757 0.026383134129218243
Accuracies: [0.7190082669258118, 0.7427386045455933, 0.7676348686218262, 0.7095435857772827, 0.7095435857772827] 0.7296937823295593 0.02252081036428156
Precisions: [0.6336633563041687, 0.6538461446762085, 0.6969696879386902, 0.605042040348053, 0.6016260385513306] 0.6382294535636902 0.03508948505301734
Recals: [0.6736842393875122, 0.7234042286872864, 0.7263157963752747, 0.75789475440979, 0.7789473533630371] 0.7320492744445801 0.03572535666722884
